
### Telco Customer Churn using ANN
#### based on examples from: <br> https://www.kaggle.com/snehithatiger/telco-customer-churn-using-ann 


In [1]:
# linear algebra
import numpy as np

# data processing
import pandas as pd 

#CSV file I/O (e.g. pd.read_csv)
import os

#plotting the graph
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib

# scki-learn library - converting the non-numeric data into numeric data.
from sklearn.preprocessing import LabelEncoder

# scki-learn library - partition the dataset for modelling
from sklearn.model_selection import train_test_split

# scki-learn library - metrics for ml models
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_recall_curve

#scki-learn library - KN Neighbors
from sklearn.neighbors import KNeighborsClassifier

#scki-learn library - Random Tree Models
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

# xgboost - used for Tree Classifier Models
from xgboost import XGBClassifier

# scki-learn library - prediction using Naive Bayes Algorithm 
from sklearn.naive_bayes import GaussianNB

# keras - tensorflow for artificial neutral networks
import keras
from keras.models import Sequential
from keras.layers import Dense




Using matplotlib backend: Qt5Agg
Using TensorFlow backend.


In [2]:
# Set the active directory to access the data
os.chdir('/home/mike/Documents/mkp_code/Institute of Data Course/telco-customer-churn-project/data/unprocessed')

In [3]:
#importing the dataset
customer_data = pd.read_csv('datasets_13996_18858_WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [4]:
# Display the first 5 rows
customer_data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
# What does the data look like?
customer_data.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [6]:
# checking if any null data exists
customer_data.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [7]:
# drop a column that is not useful
customer_data = customer_data.drop(columns = ['customerID'])


In [8]:
# Check the dataset again
customer_data.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [9]:
# converting the non-numeric data into numeric data.
encoded = customer_data.apply(lambda x: LabelEncoder().fit_transform(x) if x.dtype == 'object' else x)

In [10]:
encoded.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,1,0,1,0,1,0,0,2,0,0,0,0,0,1,2,29.85,2505,0
1,1,0,0,0,34,1,0,0,2,0,2,0,0,0,1,0,3,56.95,1466,0
2,1,0,0,0,2,1,0,0,2,2,0,0,0,0,0,1,3,53.85,157,1
3,1,0,0,0,45,0,1,0,2,0,2,2,0,0,1,0,0,42.30,1400,0
4,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,1,2,70.70,925,1


In [11]:
# X=Features, y=Target
X = encoded.iloc[:, 0:19]
y = encoded.Churn

In [12]:
# train test spliting
# split the data into a training and a test part.
# the models will be trained on the training data set and tested on the test data set.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [13]:
#print length of X_train, X_test, y_train, y_test
print ("X_train: ", len(X_train))
print("X_test: ", len(X_test))
print("y_train: ", len(y_train))
print("y_test: ", len(y_test))

X_train:  5634
X_test:  1409
y_train:  5634
y_test:  1409


In [14]:
# Model the K Neighbor
knc = KNeighborsClassifier()
knc.fit(X_train, y_train)
knc.fit(X_train, y_train)
print('Accuracy score of KNN training set: {:.3f}'.format(knc.score(X_train, y_train)))
print('Accuracy score of KNN test set: {:.3f}'.format(knc.score(X_test, y_test)))


Accuracy score of KNN training set: 0.817
Accuracy score of KNN test set: 0.765


In [15]:
# Print Results - KNN Classifier
y_knc = knc.predict(X_test)

print('confusion_matrix of KNN: ', confusion_matrix(y_test, y_knc))
print('precision_score of KNN: ', precision_score(y_test, y_knc))
print('recall_score of KNN: ', recall_score(y_test, y_knc))
print('precision_recall_curve of KNN: ', precision_recall_curve(y_test, y_knc))

confusion_matrix of KNN:  [[921 128]
 [203 157]]
precision_score of KNN:  0.5508771929824562
recall_score of KNN:  0.4361111111111111
precision_recall_curve of KNN:  (array([0.25550035, 0.55087719, 1.        ]), array([1.        , 0.43611111, 0.        ]), array([0, 1]))


In [16]:
# Model the Random Forest Regressor
rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)
print('Accuracy score (training): {:.3f}'.format(rfr.score(X_train, y_train)))
print('Accuracy score (test): {:.3f}'.format(rfr.score(X_test, y_test)))

Accuracy score (training): 0.889
Accuracy score (test): 0.252


In [17]:
# Model Random Forest Classifier
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
print('Accuracy score Random Forest Classifier training set: {:.3f}'.format(rfc.score(X_train, y_train)))
print('Accuracy score Random Forest Classifier test set: {:.3f}'.format(rfc.score(X_test, y_test)))

Accuracy score Random Forest Classifier training set: 0.997
Accuracy score Random Forest Classifier test set: 0.787


In [18]:
y_rfc = rfc.predict(X_test)

print('confusion_matrix of Random Forest Classifier: ', confusion_matrix(y_test, y_rfc))
print('precision_score of Random Forest Classifier: ', precision_score(y_test, y_rfc))
print('recall_score of Random Forest Classifier: ', recall_score(y_test, y_rfc))
print('precision_recall_curve of Random Forest Classifier: ', precision_recall_curve(y_test, y_rfc))

confusion_matrix of Random Forest Classifier:  [[938 111]
 [189 171]]
precision_score of Random Forest Classifier:  0.6063829787234043
recall_score of Random Forest Classifier:  0.475
precision_recall_curve of Random Forest Classifier:  (array([0.25550035, 0.60638298, 1.        ]), array([1.   , 0.475, 0.   ]), array([0, 1]))


In [19]:
# Model Decision Tree Classifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 42)
classifier.fit(X_train, y_train)

print('accuracy of Decision Tree Classifier training set: {:.3f}'.format(classifier.score(X_train,y_train)))
print('accuaracy of Decision Tree Classifier test set: {:.3f}'.format(classifier.score(X_test, y_test)))

accuracy of Decision Tree Classifier training set: 0.997
accuaracy of Decision Tree Classifier test set: 0.735


In [20]:
y_dtc = classifier.predict(X_test)
print('accuracy_score of decision tree classifier: ', accuracy_score(y_dtc, y_test))
print('confusion_matrix of decision tree classifier: ', confusion_matrix(y_dtc, y_test))
print('precision_score of decision tree classifier: ', precision_score(y_dtc, y_test))
print('recall_score of decision tree classifier: ', recall_score(y_dtc, y_test))
print('precision_recall_curve of decision tree classifier: ', precision_recall_curve(y_dtc, y_test))

accuracy_score of decision tree classifier:  0.7352732434350603
confusion_matrix of decision tree classifier:  [[838 162]
 [211 198]]
precision_score of decision tree classifier:  0.55
recall_score of decision tree classifier:  0.4841075794621027
precision_recall_curve of decision tree classifier:  (array([0.29027679, 0.55      , 1.        ]), array([1.        , 0.48410758, 0.        ]), array([0, 1]))


In [21]:
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
print('Accuracy XGBclassifier on train set: {:.3f}'.format(xgb.score(X_train, y_train)))
print('Accuracy XGBClassifier on test set: {:.3f}'.format(xgb.score(X_test, y_test)))

Accuracy XGBclassifier on train set: 0.945
Accuracy XGBClassifier on test set: 0.793


In [22]:
y_xgbc = xgb.predict(X_test)
# predicting Confusion matrix, accuracy score,precision score, recall score
print('accuracy_score of xgboost: ', accuracy_score(y_test, y_xgbc))
print('confusion_matrix of xgboost: ', confusion_matrix(y_test, y_xgbc))
print('precision_score of xgboost: ', precision_score(y_test, y_xgbc))
print('recall_score of xgboost: ', recall_score(y_test, y_xgbc))
print('precision_recall_curve of xgboost: ', precision_recall_curve(y_test, y_xgbc))

accuracy_score of xgboost:  0.7927608232789212
confusion_matrix of xgboost:  [[934 115]
 [177 183]]
precision_score of xgboost:  0.6140939597315436
recall_score of xgboost:  0.5083333333333333
precision_recall_curve of xgboost:  (array([0.25550035, 0.61409396, 1.        ]), array([1.        , 0.50833333, 0.        ]), array([0, 1]))


In [23]:
# prediction using Naive Bayes Algorithm 
nbc = GaussianNB()
nbc.fit(X_train, y_train)

print('accuracy of Naive Bayes training set: {:.3f}'.format(nbc.score(X_train,y_train)))
print('accuaracy of Naive Bayes test set: {:.3f}'.format(nbc.score(X_test, y_test)))

accuracy of Naive Bayes training set: 0.754
accuaracy of Naive Bayes test set: 0.752


In [24]:
y_nb = nbc.predict(X_test)

print('accuracy_score of Naive Bayes: ', accuracy_score(y_test, y_nb))
print('confusion_matrix of Naive Bayes: ', confusion_matrix(y_test, y_nb))
print('precision_score of Naive Bayes: ', precision_score(y_test, y_nb))
print('recall_score of Naive Bayes: ', recall_score(y_test, y_nb))
print('precision_recall_curve of Naive Bayes: ', precision_recall_curve(y_test, y_nb))

accuracy_score of Naive Bayes:  0.752306600425834
confusion_matrix of Naive Bayes:  [[795 254]
 [ 95 265]]
precision_score of Naive Bayes:  0.5105973025048169
recall_score of Naive Bayes:  0.7361111111111112
precision_recall_curve of Naive Bayes:  (array([0.25550035, 0.5105973 , 1.        ]), array([1.        , 0.73611111, 0.        ]), array([0, 1]))


In [25]:
# Initialising the Multi Layer Perceptron (MLP)
classifier = Sequential()

In [26]:
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = 19))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [27]:
# Adding the second hidden layer
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))

In [28]:
# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

In [29]:
# Compiling the MLP
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [30]:
# Fitting the MLP to the Training set
classifier.fit(X_train, y_train, batch_size = 5, epochs = 100)

Train on 5634 samples
Epoch 1/100
5634/5634 [==============================] - 2s 402us/sample - loss: 0.5535 - acc: 0.7355
Epoch 2/100
5634/5634 [==============================] - 2s 301us/sample - loss: 0.4854 - acc: 0.7673
Epoch 3/100
5634/5634 [==============================] - 2s 291us/sample - loss: 0.4701 - acc: 0.7746
Epoch 4/100
5634/5634 [==============================] - 2s 305us/sample - loss: 0.4639 - acc: 0.7806
Epoch 5/100
5634/5634 [==============================] - 2s 286us/sample - loss: 0.4570 - acc: 0.7797
Epoch 6/100
5634/5634 [==============================] - 2s 290us/sample - loss: 0.4571 - acc: 0.7792
Epoch 7/100
5634/5634 [==============================] - 2s 303us/sample - loss: 0.4592 - acc: 0.7824
Epoch 8/100
5634/5634 [==============================] - 2s 294us/sample - loss: 0.4501 - acc: 0.7820
Epoch 9/100
5634/5634 [==============================] - 2s 303us/sample - loss: 0.4521 - acc: 0.7765
Epoch 10/100
5634/5634 [==============================] - 2s

In [31]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                320       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 609
Trainable params: 609
Non-trainable params: 0
_________________________________________________________________


In [32]:
# prediction using Neural Networks 

print('accuracy of MLP training set: {:.3f}'.format(classifier.score(X_train,y_train)))
print('accuaracy of MLP test set: {:.3f}'.format(classifier.score(X_test, y_test)))

AttributeError: 'Sequential' object has no attribute 'score'

In [ ]:
accuracy_score(y_test, y_pred)